In [1]:
class Gentle:
    import pandas as pd
    import numpy as np
    from sklearn.ensemble import VotingClassifier, VotingRegressor
    from catboost import CatBoostClassifier, CatBoostRegressor
    from lightgbm import LGBMClassifier, LGBMRegressor
    from xgboost.sklearn import XGBClassifier, XGBRegressor
    from sklearn.model_selection import train_test_split, cross_validate, StratifiedKFold, KFold
        
    import warnings
    warnings.filterwarnings('ignore')

    def _init_(self):
        self._gentle = []

    def read_data(self, Trainfile):
        """ 
        Importing the csv files.
        Trainfile: The csv dataset to be analyzed
        """
        try:
            frain = pd.read_csv(Trainfile)
        except:
            return "Error occured when importing files"
        return frain

    def model(self, ptype, frain):
        """
        To create a model for the dataset.
        frain: the imported dataset
        ptype: problem type - classification or regression.
            for classification type 'class'
            for regression, type 'reg' 
        """
        try:
            # Splitting Target and Dependent variables
            X = frain.drop(target, axis =1)
            y = frain[target]
            
            # Splitting the datasets
            trainX,testX,trainy,testy = train_test_split(X,y, test_size =0.3, random_state = 101)
            
            models = []
            results = []
            names = []
            
            # CLASSIFICATON
            if ptype == 'class':
                # Adding algorthms
                models.append(('cat', CatBoostClassifier(verbose=0)))
                models.append(('lgbm', LGBMClassifier()))
                models.append(('xgb', XGBClassifier()))

                # evaluate -cross validation- each model in turn
                scoring =['roc_auc', 'balanced_accuracy']
                for name, model in models:
                    kfold = StratifiedKFold(n_splits=5, shuffle= True)
                    cv_results = cross_validate(model, trainX, trainy, cv=kfold, scoring=scoring, return_train_score=True)
                    results.append([cv_results['test_roc_auc'].mean(), cv_results[f'test_balanced_accuracy'].mean()])
                    names.append(name)
                alg = pd.DataFrame({'models': models, 'roc_auc':[m[0] for m in results] , 'balanced_accuracy': [m[1] for m in results]})
                vtc = VotingClassifier([n for n in alg.sort_values(metric, ascending=False)['models'][:2]], weights =[i for i in alg[metric].nlargest(2)], voting='soft')
                vtc.fit(trainX, trainy) # Fitting the model(ensemble)
                ypred = vtc.predict(test_X) # Predicting the tests
                sol = vtc.score(trainX, trainy)    # Train Scores
                sol1 = vtc.score(testX, testy)  # Test Scores

            # REGRESSION
            elif ptype == 'reg':
                # Adding algorthms
                models.append(('cat', CatBoostRegressor(verbose=0)))
                models.append(('lgbm', LGBMRegressor()))
                models.append(('xgb', XGBRegressor()))

                # evaluate -cross validation- each model in turn
                scoring =['neg_root_mean_squared_error', 'r2']
                for name, model in models:
                    kfold = StratifiedKFold(n_splits=5, shuffle= True)
                    cv_results = cross_validate(model, trainX, trainy, cv=kfold, scoring=scoring, return_train_score=True)
                    results.append([-cv_results['test_neg_root_mean_squared_error'].mean(), cv_results[f'test_r2'].mean()])
                    names.append(name)
                alg = pd.DataFrame({'models': models, 'RMSE':[m[0] for m in results] , 'r2': [m[1] for m in results]})
                vtr = VotingRegressor([n for n in alg.sort_values('RMSE', ascending=True)['models']][:2], weights =[i for i in alg[metric].nsmallest(2)], voting = 'hard')
            vtr.fit(trainX, trainy) # Fitting the model (ensemble)
            ypred = vtr.predict(test_X)  # Predicting the tests
            sol = vtr.score(trainX, trainy)    # Train Scores
            sol1 = vtr.score(testX, testy) # Test Scores
        except:
            return f"Error occured when modeling for {ptype}"

        Scores = f'Ensemble: The Train score is {sol}, The Test score is {sol1}'
        return alg, Scores

ModuleNotFoundError: No module named 'catboost'